In [0]:
## let us import necessary libraries ##

In [0]:
%sh
pip install tweepy

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import tweepy
from tweepy import Stream
##from tweepy.streaming import StreamListener##
from tweepy import OAuthHandler
import socket
import json

In [0]:
## assigning necessary twitter access tokens ##

In [0]:
api_key =       "OGWqLKRyUqdo6NTwp07x2yQhc"
api_secret =    "wZaNMcXsvX43aQ0rh2ziBxqxn0sXtetIFQlBltHntUItHl7VTX"
access_token =  "1389687634813980675-eIftIGC1SilkZz1hwjbvZQc8hd5TaN"
access_secret = "4DLtLTnW97kSH8G4J1aGBMismYfpaMC8WO1GBQcgXCRCS"

In [0]:
## creation of socket ##

In [0]:
class TweetsListener(tweepy.Stream):
  # tweet object listens for the tweets
  def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, csocket):
    super(TweetsListener, self).__init__(consumer_key, consumer_secret, access_token, access_token_secret)
    self.client_socket = csocket
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text']+"t_end")\
            .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['text']+"t_end")\
            .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True

  #def on_status(self,)

  def on_error(self, status):
    print(status)
    return True
  pass

def sendData(c_socket, keyword):
  print('start sending data from Twitter to socket')
  # authentication based on the credentials
  auth = OAuthHandler(api_key, api_secret)
  auth.set_access_token(access_token, access_secret)
  api = tweepy.API(auth)
  # start sending data from the Streaming API 
  twitter_stream = TweetsListener(api_key, api_secret, access_token, access_secret, c_socket)
  twitter_stream.filter(track = keyword, languages=["en"])

ERROR:tweepy.streaming:Stream encountered HTTP error: 401


In [0]:
# server (local machine) creates listening socket
s = socket.socket()
s.settimeout(10)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
host = "0.0.0.0"
port = 33333

import sys
try:
    s.bind((host, port))
except socket.error as msg:
    print("Couldnt connect with the socket-server: %s \n terminating program" % msg)
    sys.exit(1)

print("socket bind complete")
# server (local machine) listens for connections
s.listen(5)
print('socket is listening')


# return the socket and the address on the other side of the connection (client side)
# https://stackoverflow.com/questions/34147424/cant-get-return-value-after-socket-accept/34149922
# firewall settings?
# try pinging?
#import select
#readable, writable, exceptional = select.select([s], [], [s])

#print(readable)
#print(writable)
#print(exceptional)
'''
try:
    c, addr = s.accept()
except socket.error as msg:
    print("Couldnt accept: %s \n terminating program" % msg)
    sys.exit(1)
'''
#c_socket, addr = s.accept()

# try a thread instead?

def StartSocket():
    global c, addr
    
    try:
        c, addr = s.accept()
        print("Received request from: " + str(addr))
        sendData(c, keyword = ['music'])
    except socket.error as msg:
        print("Couldnt accept: %s \n terminating program" % msg)
        sys.exit(1)

import threading
thread = threading.Thread(target=StartSocket) #fork off the server code
thread.start()


#print("Received request from: " + str(addr))

# select here the keyword for the tweet data
#sendData(c_socket, keyword = ['music'])

socket bind complete
socket is listening


In [0]:
! date

#echo reset system clock to prevent twitter HTTP 401 errors

Tue Nov 23 21:51:43 UTC 2021


In [0]:
# select here the keyword for the tweet data


In [0]:
tweet_df = spark \
    .readStream \
    .format("socket") \
    .option("host", host) \
    .option("port", port) \
    .load()

# Start running the query that prints the running counts to the console
query = tweet_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

Received request from: ('127.0.0.1', 40126)
start sending data from Twitter to socket
new message
RT @scarletmoon_: Nier Replicant ver.1.22474487139 feat music by Keiichi Okabe and vocals by Emi Evans has been nominated for Best Score an…
new message
RT @wantsyouz: GRAMMY NOMINATED SONG + REAL MUSIC  https://t.co/WN4Fq7uEBg
new message
RT @BTSArmyZA: [LOCAL NEWS]📰

There are multiple news headlines across South Africa on @BTS_twt' historic AMA wins!🇿🇦

1. https://t.co/OtNG…
new message
Omg I remember when I first watched this interview I was crying with her!!! Lana and Marina will always have my heart no matter what. The way they changed not only the music industry and culture but also so many peoples lives forever ❤️🧡💚💙💜 https://t.co/6R5AZoV4J4
new message
@RecordingAcad what is wrong with y’all https://t.co/bV4CLEI7Fc
new message
RT @shysmith: “Soaked” is out tonight at midnight &lt;3 
This is a thank you to everyone who’s waited patiently for music and has stuck by my…
new message
R

In [0]:

# type cast the column value
tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")


wordCounts = words.groupBy("word").count()

In [0]:
 # Start running the query that prints the running counts to the console
query = tweet_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

In [0]:
writeTweet = tweet_df_string.writeStream. \
    outputMode("complete"). \
    format("memory"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()

In [0]:
%sql 
select *  
from tweetquery
limit 100

In [0]:
%sh
pip install TextBlob

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

In [0]:
def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words

In [0]:
## creation of spark session ##

In [0]:
if __name__ == "__main__":
    
    spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()
    lines = spark.readStream.format("socket").option("host", "0.0.0.0").option("port", 5555).load()
    words = preprocessing(lines)